In [2]:
import sys
import subprocess
import importlib

#packages = ['pandas', 'numpy','pathlib','os','glob', 'textblob']
#[subprocess.check_call(['pip', 'install', pkg]) 
#for pkg in packages if not importlib.util.find_spec(pkg)]
import pandas as pd
import numpy as np

import datetime as dt
from pathlib import Path
import pathlib
import os
import glob
from datetime import datetime as dt
from textblob import TextBlob
from datetime import timedelta 


In [3]:
campaigns=pd.read_csv("./Data/campaigns.csv",delimiter=";")

for i in range(0,len(campaigns)):
    campaigns.iloc[i,2]=campaigns.iloc[i,2].replace('\u202f','')

for i in range(0,len(campaigns)):
    campaigns.iloc[i,3]=campaigns.iloc[i,3].replace('\xa0€','')

for i in range(0,len(campaigns)):
    campaigns.iloc[i,3]=campaigns.iloc[i,3].replace(',','.')



campaigns['date']=pd.to_datetime(campaigns['date'])
campaigns['lettersSent']=campaigns['lettersSent'].astype(int)
campaigns['lettersSent']=campaigns['lettersSent'].astype(float)
campaigns.head()


,campaignID,date,lettersSent,CostUnit
0,153,2004-12-13,6873.0,0.17
1,154,2005-01-01,7656.0,0.30
2,180,2005-01-18,9933.0,0.20
3,433,2005-01-29,7448.0,0.59
4,476,2005-12-02,6605.0,0.51


In [66]:
campaigns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   campaignID   367 non-null    int64         
 1   date         367 non-null    datetime64[ns]
 2   lettersSent  367 non-null    float64       
 3   CostUnit     367 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 11.6+ KB


In [68]:
campaigns[(campaigns['campaignID']==6169)|
(campaigns['campaignID']==7244)|
(campaigns['campaignID']==7362)]

,campaignID,date,lettersSent,CostUnit
335,6169,2018-04-09,34888.0,0.87
354,7244,2019-06-18,25645.0,0.55


In [74]:
campaigns[campaigns['date']>'2019-06-18']

,campaignID,date,lettersSent,CostUnit
355,7276,2019-07-07,5409.0,0.31
356,7330,2019-07-17,5391.0,0.02
358,7346,2019-08-25,15304.0,0.61
359,7356,2019-08-09,9619.0,0.26
360,7395,2019-09-18,38629.0,0.01
361,7408,2019-09-29,6826.0,0.28
362,7494,2019-10-17,8539.0,0.14
363,7503,2019-10-29,6126.0,0.15
364,7505,2019-11-16,29746.0,0.12
365,7519,2019-11-26,35199.0,0.36


In [17]:
donors=pd.read_csv("donors.csv")

In [18]:
donors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44691 entries, 0 to 44690
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   44691 non-null  int64  
 1   donorID      44691 non-null  int64  
 2   zipcode      44689 non-null  float64
 3   province     44691 non-null  object 
 4   region       44691 non-null  object 
 5   gender       44659 non-null  object 
 6   language     44691 non-null  object 
 7   dateOfBirth  44691 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 2.7+ MB


In [19]:
donors.isna().sum()

Unnamed: 0      0
donorID         0
zipcode         2
province        0
region          0
gender         32
language        0
dateOfBirth     0
dtype: int64

In [4]:
gifts=pd.read_csv("./Data/gifts.csv")

gifts.loc[gifts['campaignID'].isna(),['campaignID']]=0
#gifts=gifts[gifts['campaignID']!=0]
gifts['campaignID']=gifts['campaignID'].astype(int)
gifts['date']=pd.to_datetime(gifts['date'])
gifts['amount']=gifts['amount'].str.replace(',','.')
gifts['amount']=gifts['amount'].astype(float)

#subseting the data for 6169 campaign
gifts6169=gifts[gifts['date']<='2018-09-24 00:00:00']
date = dt(year=2018, day=24, month=9)
#calculate the diference between the refdate and transaction date
gifts6169['date_1']=date - gifts6169['date']
#converting the type of duration to integer based on number of days
gifts6169['date_1']=gifts6169['date_1'].dt.days.astype(int)
def agg(tr): 
    
    tr['amount_avg'] = tr['amount'].mean()
    tr['freq_overal'] = tr['donorID'].count()
    tr['freq_campagin'] = tr.loc[tr['campaignID']!=0].count()
    tr['freq_no_camp'] = tr.loc[tr['campaignID']==0].count()
    tr['date_first'] = tr['date'].min()
    tr['date_last'] = tr['date'].max()
    tr['date_1'] = tr['date_1'].mean()
    
    #credit = tr.loc[(tr['trans_type'] == 'Credit')]
    #withdrawal = tr.loc[(tr['trans_type'] == 'Withdrawal')]
    
    #tr['trans_cred'] = credit['trans_amount'].mean()
    #tr['trans_withdrawal'] = withdrawal['trans_amount'].mean()
    return tr
gifts6169 = gifts6169.groupby(by='donorID').apply(agg)
#subseting the data for 7244 campaign
gifts7244=gifts[gifts['date']<='2019-05-18 00:00:00']
#subseting the data for 7362 campaign
gifts7362=gifts[gifts['date']<='2020-01-01 00:00:00']


gifts.head()

<ipython-input-4-4962ebe82f64>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gifts6169['date_1']=date - gifts6169['date']
<ipython-input-4-4962ebe82f64>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gifts6169['date_1']=gifts6169['date_1'].dt.days.astype(int)


,Unnamed: 0,donorID,campaignID,amount,date
0,0,100001,0,12.39,2004-03-23
1,1,100001,1577,12.39,2007-10-11
2,2,100001,1634,6.69,2007-12-28
3,3,100001,1778,24.79,2008-06-14
4,4,100001,1808,7.44,2008-07-09


In [6]:
x=gifts[gifts['campaignID']==0]

In [7]:
x

,Unnamed: 0,donorID,campaignID,amount,date
0,0,100001,0,12.39,2004-03-23
29,29,100004,0,24.79,2006-03-30
57,57,100007,0,24.79,2006-09-04
112,112,100020,0,24.79,2001-12-05
181,181,100025,0,20.00,2009-05-29
...,...,...,...,...,...
216506,216506,144666,0,32.25,2010-10-11
216507,216507,144667,0,49.58,2004-02-01
216514,216514,144668,0,20.00,2019-11-20
216560,216560,144683,0,50.00,2015-10-11


In [110]:
date = dt(year=2018, day=24, month=9)

TypeError: 'module' object is not callable

In [89]:
gifts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216594 entries, 0 to 216593
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Unnamed: 0  216594 non-null  int64         
 1   donorID     216594 non-null  int64         
 2   campaignID  216594 non-null  int32         
 3   amount      216594 non-null  float64       
 4   date        216594 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 7.4 MB


In [101]:
gifts6169.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 209829 entries, 0 to 216593
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Unnamed: 0  209829 non-null  int64         
 1   donorID     209829 non-null  int64         
 2   campaignID  209829 non-null  int32         
 3   amount      209829 non-null  float64       
 4   date        209829 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 8.8 MB


In [102]:
gifts7244.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213388 entries, 0 to 216593
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Unnamed: 0  213388 non-null  int64         
 1   donorID     213388 non-null  int64         
 2   campaignID  213388 non-null  int32         
 3   amount      213388 non-null  float64       
 4   date        213388 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 9.0 MB


In [103]:
gifts7362.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216586 entries, 0 to 216593
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Unnamed: 0  216586 non-null  int64         
 1   donorID     216586 non-null  int64         
 2   campaignID  216586 non-null  int32         
 3   amount      216586 non-null  float64       
 4   date        216586 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 9.1 MB


In [9]:
gifts6169[gifts6169['freq_overal']==0]

,Unnamed: 0,donorID,campaignID,amount,date,date_1,amount_avg,freq_overal,freq_campagin,freq_no_camp,date_first,date_last


In [106]:
gifts7362.groupby('donorID')['date'].min().max()

Timestamp('2018-09-23 00:00:00')

In [ ]:
gifts.groupby('donorID')['date'].count()

donorID
100001     12
100002      1
100003     12
100004     26
100005      5
           ..
265584      1
281770      1
317880      1
480980      1
1474536     1
Name: date, Length: 44700, dtype: int64

In [ ]:
train=pd.read_csv("./Data/selection campaign 6169.csv")

In [ ]:
camp6169=pd.merge(train,gifts,on='donorID',how='left')

camp6169.isna().sum()


donorID       0
Unnamed: 0    5
campaignID    5
amount        5
date          5
dtype: int64

In [ ]:
camp6169=pd.merge(camp6169,donors,on='donorID',how='left')
camp6169.isna().sum()


donorID           0
Unnamed: 0_x      5
campaignID        5
amount            5
date              5
Unnamed: 0_y     14
zipcode          27
province         14
region           14
gender          121
language         14
dateOfBirth      14
dtype: int64

In [ ]:
d=dt.date(year=2018,month=9,day=4)-dt.timedelta(days=190)
d

datetime.date(2018, 2, 26)

In [ ]:
all6169=gifts[gifts['date']<'2018-09-04']

In [ ]:
all6169.groupby('donorID')['date'].min().min()

Timestamp('2000-06-05 00:00:00')

In [ ]:
camp6169=camp6169[(camp6169['date']<'2018-09-04 00:00:00')]#&(camp6169['date']>'2018-02-26')]

In [ ]:
camp6169.groupby('donorID')['date'].max().max()

Timestamp('2018-09-03 00:00:00')

In [ ]:
test=pd.read_csv("./Data/selection campaign 7244.csv")

In [ ]:
test

,donorID
0,100001
1,100002
2,100008
3,100009
4,100010
...,...
25640,144682
25641,144683
25642,144686
25643,144690


In [ ]:
all7244=gifts[gifts['date']<'2019-06-18']


In [ ]:
all7244.groupby('donorID')['date'].min().min()

Timestamp('2000-06-05 00:00:00')

In [ ]:
camp7244=pd.merge(test,gifts,on='donorID',how='left')

In [ ]:
camp7244=camp7244[(camp7244['date']<'2019-06-18 00:00:00')]#&(camp6169['date']>'2018-02-26')]

In [ ]:
camp7244.groupby('donorID')['date'].max().max()

Timestamp('2018-09-03 00:00:00')